In [1]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.0/608.0 kB 6.4 MB/s eta 0:00:00


In [2]:
import torch
import cv2
import os
from ultralytics import YOLO

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
def load_yolov5_model(weights_path):
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
    return model

In [5]:
def draw_bounding_boxes(label_file, results):
    image = cv2.imread("/content/drive/MyDrive/Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg")
    boxes = results.xyxy[0].detach().cpu().numpy()

    with open(label_file, 'w') as f:
      for i,box in enumerate(boxes):
        im=image.copy()
        x1, y1, x2, y2 = box[:4].astype(int)
        coord=(x1,y1,x2,y2)
        im=im[y1:y2,x1:x2]
        # os.chdir(image_path)
        # cv2.imwrite(f"img{i}.jpg",im)
        # os.chdir("/content/drive")
        print(coord)
        label = f"{results.names[int(box[5])]}"
        f.write(f"{x1} {y1} {x2} {y2}\n")

In [6]:
def load_model(model_path):
    model=YOLO(model=model_path)
    return model

In [7]:
def extract_roi(image_path,coord_list,i,j):
    image=cv2.imread(image_path)
    im=image.copy()

    x1, y1, x2, y2 = coord_list
    roi = im[y1:y2, x1:x2]

    cv2.imwrite(f"roi{j}{i}.jpg",roi)
    return roi

In [14]:
def extract_labels(roi,model,i,j):



    results=model.predict(source=f"roi{j}{i}.jpg",conf=0.9,save=True,save_txt=True,show_labels=False)
    contours=[]
    for result in results:
        boxes = result.boxes

    i = 0
    label_string_list =[]
    num_objects=0
    num_emptys=0
    while i < len(boxes.xyxy.tolist()):

      bbox = boxes.xyxy.tolist()[i]
      class_id = int(boxes.cls.tolist()[i])
      if class_id==0:
        num_objects +=1
      else:
        num_emptys +=1
      i+=1
    return num_objects,num_emptys
    print("Objects:",num_objects)
    print("emptys:",num_emptys)

In [17]:
weights_path = "/content/drive/MyDrive/phase_2.pt"
model = load_yolov5_model(weights_path)
model_path="/content/drive/MyDrive/latest_detection_model.pt"
image_list=["/content/drive/MyDrive/1663135227502.62cd7c06be8154e6d5c866b4.jpg","/content/drive/MyDrive/Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg"]
parent_list=[]
model1=load_model(model_path)
for j in range(len(image_list)):

  image=cv2.imread(image_list[j])

  results=model(image)
  label_file="/content/drive/MyDrive/data/output{j}"
  draw_bounding_boxes(label_file, results)
  results_dict={}
  output_path=label_file
  bounding_box_list=[]
  with open(output_path, 'r') as f:
    for line in f:
      bounding_box = [int(coord) for coord in line.strip().split()]
      bounding_box_list.append(bounding_box)
  destination_path = image_list[j]
  results_dict["image_name"] = destination_path.split("/")[-1]
  results_dict["image_id"] = j
  results_dict["coordinates"]=bounding_box_list
  counts_list=[]


  for i in range(len(bounding_box_list)):
    roi = extract_roi(destination_path,bounding_box_list[i],i,j)
    num_objs,num_empts=extract_labels(roi,model1,i,j)
    counts_list.append([num_objs,num_empts])
  results_dict["count"] = counts_list
  parent_list.append(results_dict)
print(parent_list)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-26 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20856975 parameters, 0 gradients
Adding AutoShape... 


(26, 0, 1642, 988)
(3170, 288, 4381, 1190)
(1704, 19, 3109, 1059)
(1699, 1075, 3153, 2157)
(1683, 2215, 3159, 3339)
(3176, 1240, 4516, 2193)
(3239, 2416, 4608, 3352)
(3, 2332, 1517, 3396)
(0, 2321, 1545, 3410)
(3234, 2415, 4605, 3347)



image 1/1 /content/roi00.jpg: 416x640 (no detections), 36.4ms
Speed: 12.3ms preprocess, 36.4ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict6
0 label saved to runs/detect/predict6/labels

image 1/1 /content/roi01.jpg: 480x640 20 objects, 1 empty, 29.3ms
Speed: 4.0ms preprocess, 29.3ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict6
1 label saved to runs/detect/predict6/labels

image 1/1 /content/roi02.jpg: 480x640 15 objects, 28.3ms
Speed: 3.7ms preprocess, 28.3ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict6
2 labels saved to runs/detect/predict6/labels

image 1/1 /content/roi03.jpg: 480x640 20 objects, 28.2ms
Speed: 2.6ms preprocess, 28.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict6
3 labels saved to runs/detect/predict6/labels

image 1/1 /content/roi04.jpg: 

(2859, 334, 4396, 1304)
(782, 1769, 1607, 2196)
(2875, 1448, 4471, 2443)
(2881, 2654, 4582, 3456)
(694, 2623, 1553, 3123)
(791, 705, 1634, 1487)
(1651, 1731, 2786, 2234)
(1, 2382, 613, 3039)
(1600, 2673, 2798, 3332)
(3, 1650, 718, 2250)
(70, 860, 771, 1534)



image 1/1 /content/roi10.jpg: 416x640 4 objects, 28.7ms
Speed: 2.4ms preprocess, 28.7ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict6
10 labels saved to runs/detect/predict6/labels

image 1/1 /content/roi11.jpg: 352x640 (no detections), 26.2ms
Speed: 1.7ms preprocess, 26.2ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs/detect/predict6
10 labels saved to runs/detect/predict6/labels

image 1/1 /content/roi12.jpg: 416x640 2 objects, 29.4ms
Speed: 2.2ms preprocess, 29.4ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict6
11 labels saved to runs/detect/predict6/labels

image 1/1 /content/roi13.jpg: 320x640 3 objects, 23.3ms
Speed: 1.9ms preprocess, 23.3ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)
Results saved to runs/detect/predict6
12 labels saved to runs/detect/predict6/labels

image 1/1 /content/roi14.jpg: 384x640

[{'image_name': '1663135227502.62cd7c06be8154e6d5c866b4.jpg', 'image_id': 0, 'coordinates': [[26, 0, 1642, 988], [3170, 288, 4381, 1190], [1704, 19, 3109, 1059], [1699, 1075, 3153, 2157], [1683, 2215, 3159, 3339], [3176, 1240, 4516, 2193], [3239, 2416, 4608, 3352], [3, 2332, 1517, 3396], [0, 2321, 1545, 3410], [3234, 2415, 4605, 3347]], 'count': [[0, 0], [20, 1], [15, 0], [20, 0], [11, 0], [23, 0], [1, 0], [18, 0], [19, 0], [1, 0]]}, {'image_name': 'Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg', 'image_id': 1, 'coordinates': [[2859, 334, 4396, 1304], [782, 1769, 1607, 2196], [2875, 1448, 4471, 2443], [2881, 2654, 4582, 3456], [694, 2623, 1553, 3123], [791, 705, 1634, 1487], [1651, 1731, 2786, 2234], [1, 2382, 613, 3039], [1600, 2673, 2798, 3332], [3, 1650, 718, 2250], [70, 860, 771, 1534]], 'count': [[4, 0], [0, 0], [2, 0], [3, 0], [2, 0], [2, 0], [0, 0], [5, 0], [3, 0], [3, 0], [4, 0]]}]
